In [6]:
import numpy as np 

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

In [7]:
#run only if geopy is not installed
#!pip install geopy
#!pip install folium

#### import the dataset

In [8]:
df = pd.read_csv('dataset/toronto_neighborhood.csv')
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
4,M9A,Queen's Park,Not assigned,43.667856,-79.532242


#### Use geopy library to get the latitude and longitude values of New York City.

In [43]:
def plotOnMap(df):
    latitude = 43.728320
    longitude = -79.411735
    map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.5)

    # add markers to map
    for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_toronto)  

    return map_toronto

In [44]:
plotOnMap(df)

In [30]:
df['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           11
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

### Lets explore the Neighborhoods of Toronto


**selecting the rows with substring "Toronto" in Borogh**

In [33]:
toronto_df = df[df['Borough'].str.contains("Toronto")]
toronto_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
8,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
10,M4E,East Toronto,The Beaches,43.676357,-79.293031
11,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
15,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
16,M6G,Downtown Toronto,Christie,43.669542,-79.422564
26,M4M,East Toronto,Studio District,43.659526,-79.340923
29,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
30,M5N,Central Toronto,Roselawn,43.711695,-79.416936


In [45]:
plotOnMap(toronto_df)

#### Getting point of interest using FourSquare API

In [50]:
CLIENT_ID = '5M40ULMEDEGPYJBQN3Z0RFOU3VZEG1FVHFHF5TP2OVJGVTLU' # your Foursquare ID
CLIENT_SECRET = 'HGUYAXIGANKU3JF3IIIEYV30FDQXS4EB10FTNYKA3HFXOQ2W' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5M40ULMEDEGPYJBQN3Z0RFOU3VZEG1FVHFHF5TP2OVJGVTLU
CLIENT_SECRET:HGUYAXIGANKU3JF3IIIEYV30FDQXS4EB10FTNYKA3HFXOQ2W


In [51]:
def getNearbyVenues(names, latitudes, longitudes, radius=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venue = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Harbourfront
